In [ ]:
import sys
import os

# מוסיף את תיקיית השורש של הפרויקט לנתיב החיפוש של פייתון
sys.path.append(os.path.abspath(os.path.join('..')))

from sigmoids_model import SigmoidsModel
from simulation.generator import SyntheticDataGenerator
import numpy as np

def gmm_length_dist(n):
    """
    מגרילה n אורכי פרגמנטים מתוך GMM של 3 גאוסיאנים
    """
    # פרמטרים מהתמונה
    mus = [175.2, 321.5, 464.1]
    sigmas = [18.4, 50.2, 21.2]
    weights = [0.55, 0.40, 0.05] # תיקון קטן לסכום של 1.0 (במקום 0.04 בתמונה)
    
    # 1. הגרלה לאיזה רכיב שייך כל פרגמנט
    components = np.random.choice(len(weights), size=n, p=weights)
    
    # 2. הגרלת האורכים מתוך הגאוסיאניות המתאימות
    lengths = np.zeros(n)
    for i in range(len(weights)):
        mask = (components == i)
        count = np.sum(mask)
        if count > 0:
            lengths[mask] = np.random.normal(mus[i], sigmas[i], count)
            
    # החזרה של מספרים שלמים וחיוביים בלבד
    return np.clip(lengths, 1, 1000).astype(int)


# הגדרת פונקציית הלמדא האמיתית לבדיקה (המודל שגילית קודם)
def true_lambda_function(l):
    # פונקציה שמדמה את הגרף שלך: עליה ב-170, יציבות, וירידה אחרי 400
    A, C, x1, k1, x2, k2 = 0.29, 0.03, 172.8, 22.1, 534.3, 71.2
    def sig(t): return 1 / (1 + np.exp(-t))
    return C + A * sig((l - x1) / k1) * sig((x2 - l) / k2)

# ייצור הדאטא
generator = SyntheticDataGenerator(
    N=200000,           # מספר מולקולות ייחודיות בספריה
    depth_c=0.5,        # עומק ריצוף MN רצוי
    length_dist_func=gmm_length_dist,
    lambda_func=true_lambda_function
)

# ייצור הקובץ
generator.simulate_sequencing("validation_test.tagAlign.gz")

# הרצת המודל על הדאטא שנוצר
model = SigmoidsModel("validation_test.tagAlign.gz")

# הצגת הגרף שמוכיח את התיקון
GMM_PARAMS = [(175.2, 18.4, 0.55), (321.5, 50.2, 0.40), (464.1, 21.2, 0.05)]
model.plot_validation_experiment(gmm_params=GMM_PARAMS)

ModuleNotFoundError: No module named 'sigmoids_model'